# Economic assumptions
### Table of Contents

* [1. Introduction ](#1.introduction)
* [2. Loading Economic assumption and computing annualized costs (marginal and fixed)](#2.Loading)
    * [2.2 General economic assumptions](#2.1.Loading)
    * [2.2 Production economic assumptions](#2.2.Loading)
    * [2.3 Compute variable and fixed annualized costs](#2.3.Loading)
* [3. Generation of assumption files for the planing/operation tool](#3.Loading)
    * [3.1 Technical on the electric mix. ](#3.1.Loading)
    * [3.2 Generating file for operation ](#3.2.operation)
    * [3.3 Generating file for planing ](#3.3.Planing)
* [4. A first evaluation of System Cost](#4.SystemCost)
    * [4.1 System cost computation](#4.1SystemCost)
* [5. Multi-zone case](#5.multizone)
    * [5.1 Production data](#5.1.multizone)
    * [5.2 Multizone Operation](#5.2.multizone)
    * [5.3 Multizone Planing](#5.3.multizone)


## 1. Introduction  <a class="anchor" id="1.introduction"></a>
In this file we learn how to compute a set of annualized costs that are used in the planing tool
For each technology it is only two numbers :
 - variable cost [€/MWh]
 - fixed annualized [€/kW/year]

These are obtained from a set of general economic assumptions
 - discounting rate [%]
 - carbon tax [€/t]

and assumptions per production technology (eventually per area)
 - CAPEX [€/kW]
 - Cost of dismantling [€/kW]
 - fixed operation and maintenance cost [€/kW/year]
 - variable cost [€/MWh]
 - life length [year]
 - construction length [year]
 - CO2 Emissions [gCO2/kWh]


Before you start here you should be familiar with annualized cost of electricity (variable and fixed cost) and levelised cost of energy.
In [my post here on lcoe](https://www.energy-alternatives.eu/2020/08/20/decomposition-lcoe.html) you will find all sources for
formulae and economic assumptions (in french, if there are people that would be willing to contribute to a translation that would be great !!)
and also [this one on the role of power and enery in system cost](https://www.energy-alternatives.eu/2020/05/07/mix-de-production-delectricite-energie-et-puissance.html).
An old excel file [here](https://docs.google.com/spreadsheets/d/1wWWu0YwsvkzmEG8XDRnsIoEegmALqZL-9XPxWx3z2Qo/edit?usp=sharing) contains similar stuff.
Side remark : to make all this be interactive in jupyter notebook, I'm using package [qgrid](https://github.com/quantopian/qgrid).

In [216]:
#region importation of modules
import os
if os.path.basename(os.getcwd())=="BasicFunctionalities":
    os.chdir('..') ## to work at project root  like in any IDE
InputFolder='Data/input/'
import pandas as pd
import qgrid # great package https://github.com/quantopian/qgrid
import numpy as np
#endregion

## 2. Loading Economic assumption and computing annualized costs (marginal and fixed) <a class="anchor" id="2.Loading"></a>

### 2.2 General economic assumptions <a class="anchor" id="2.1.Loading"></a>

In [217]:
#region - general economic assumption
General = pd.read_csv(InputFolder+'GeneralEconomicAssumptions.csv',sep=';',decimal=',',skiprows=0,comment="#")
col_def = {}#  { 'A': { 'editable': True } }
qgrid_widget = qgrid.show_grid(General, show_toolbar=True, column_definitions=col_def)
qgrid_widget
#endregion

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [218]:
General=qgrid_widget.get_changed_df()
General



,discountPercent,carbonTaxEurosPerT
0,4,40


### 2.2 Production economic assumptions  <a class="anchor" id="2.2.Loading"></a>

In [219]:
#region - general economic assumption
Production = pd.read_csv(InputFolder+'ProductionEconomicAssumptions.csv',sep=';',decimal=',',skiprows=0,comment="#")
Production=Production[Production["AREAS"]=="FR"]
qgrid_widget = qgrid.show_grid(Production, show_toolbar=True, column_definitions=col_def)
qgrid_widget

#endregion

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [220]:
Production=qgrid_widget.get_changed_df()
Production

,YEAR,AREAS,TECHNOLOGIES,CAPEX,dismantling,Variable,FOM,LL,CL,CO2Emission
0,2019,FR,TAC,600,100,60,36,30,2,600
1,2019,FR,Coal,1100,100,40,36,30,2,1000
2,2019,FR,CCG,900,100,50,36,30,2,470
3,2019,FR,OldNuke,0,1000,7,150,60,0,0
4,2019,FR,NukeCarrene,1587,1000,7,200,60,0,0
5,2019,FR,NewNuke,4000,1000,7,200,60,10,0
6,2019,FR,WindOnShore,1300,50,0,45,30,2,0
7,2019,FR,WindOffShore,2500,100,0,114,30,3,0
8,2019,FR,HydroLake,1500,10,0,20,80,10,0
9,2019,FR,HydroRiver,1500,10,0,20,80,10,0


### 2.3 Compute variable and fixed annualized costs <a class="anchor" id="2.3.Loading"></a>
For each technology it is only two numbers :
 - variable cost [€/MWh]
 - fixed annualized [€/kW/year]

The formula used here is explained [in this post](https://www.energy-alternatives.eu/2020/08/20/decomposition-lcoe.html)

In [221]:
r=(General.discountPercent[0]/100)
Production=Production.assign(
    LLr = round((1+r)/r*(1-(1+r)**(-Production['LL'])),2),
    capacityCost = round(Production.CAPEX/Production.LL+ Production.dismantling/(1+r)**(Production.LL) +Production.FOM,2),
    energyCost = round(Production.Variable + Production.CO2Emission*General.carbonTaxEurosPerT[0]/10**6,2)
)

Production
## Storage economic assumptions



,YEAR,AREAS,TECHNOLOGIES,CAPEX,dismantling,Variable,FOM,LL,CL,CO2Emission,LLr,capacityCost,energyCost
0,2019,FR,TAC,600,100,60,36,30,2,600,17.98,86.83,60.02
1,2019,FR,Coal,1100,100,40,36,30,2,1000,17.98,103.50,40.04
2,2019,FR,CCG,900,100,50,36,30,2,470,17.98,96.83,50.02
3,2019,FR,OldNuke,0,1000,7,150,60,0,0,23.53,245.06,7.00
4,2019,FR,NukeCarrene,1587,1000,7,200,60,0,0,23.53,321.51,7.00
5,2019,FR,NewNuke,4000,1000,7,200,60,10,0,23.53,361.73,7.00
6,2019,FR,WindOnShore,1300,50,0,45,30,2,0,17.98,103.75,0.00
7,2019,FR,WindOffShore,2500,100,0,114,30,3,0,17.98,228.17,0.00
8,2019,FR,HydroLake,1500,10,0,20,80,10,0,24.87,39.18,0.00
9,2019,FR,HydroRiver,1500,10,0,20,80,10,0,24.87,39.18,0.00


## 3. Generation of assumption files for the planing/operation tool <a class="anchor" id="3.Loading"></a>
### 3.1 Technical on the electric mix. <a class="anchor" id="3.1.Loading"></a>

In [222]:
#region - general economic assumption
ProductionTech = pd.read_csv(InputFolder+'ProductionTechnicalAssumptions.csv',sep=';',decimal=',',skiprows=0,comment="#")
ProductionTech=ProductionTech[ProductionTech["AREAS"]=="FR"]
qgrid_widget = qgrid.show_grid(ProductionTech, show_toolbar=True, column_definitions=col_def)
qgrid_widget
ProductionTech=qgrid_widget.get_changed_df()
ProductionTechPlus=ProductionTech.merge(Production, how='inner', left_on=["AREAS","TECHNOLOGIES"], right_on=["AREAS","TECHNOLOGIES"])
ProductionTechPlus
#endregion

,YEAR_x,AREAS,TECHNOLOGIES,capacity,ProducedEnergy,EnergyNbhourCap,_RampConstraintPlus,_RampConstraintMoins,RampConstraintPlus2,RampConstraintMoins2,...,CAPEX,dismantling,Variable,FOM,LL,CL,CO2Emission,LLr,capacityCost,energyCost
0,2019,FR,TAC,4000.0,3.2,0.0,0.0,0.0,0,0,...,600,100,60,36,30,2,600,17.98,86.83,60.02
1,2019,FR,Coal,2000.0,1.0,0.0,0.1,0.1,0,0,...,1100,100,40,36,30,2,1000,17.98,103.50,40.04
2,2019,FR,CCG,12000.0,38.6,0.0,0.1,0.1,0,0,...,900,100,50,36,30,2,470,17.98,96.83,50.02
3,2019,FR,OldNuke,63000.0,413.0,0.0,0.1,0.1,0,0,...,0,1000,7,150,60,0,0,23.53,245.06,7.00
4,2019,FR,NukeCarrene,0.0,0.0,0.0,0.0,0.0,0,0,...,1587,1000,7,200,60,0,0,23.53,321.51,7.00
5,2019,FR,NewNuke,0.0,0.0,0.0,0.0,0.0,0,0,...,4000,1000,7,200,60,10,0,23.53,361.73,7.00
6,2019,FR,WindOnShore,14900.0,29.0,0.0,0.0,0.0,0,0,...,1300,50,0,45,30,2,0,17.98,103.75,0.00
7,2019,FR,WindOffShore,0.0,0.0,0.0,0.0,0.0,0,0,...,2500,100,0,114,30,3,0,17.98,228.17,0.00
8,2019,FR,HydroLake,12500.0,33.0,3638.1,0.0,0.0,0,0,...,1500,10,0,20,80,10,0,24.87,39.18,0.00
9,2019,FR,HydroRiver,7500.0,30.0,0.0,0.0,0.0,0,0,...,1500,10,0,20,80,10,0,24.87,39.18,0.00


Assumptions on installed power and produced energy

### 3.2 Generating file for Operation <a class="anchor" id="3.2.operation"></a>

In [223]:
SelectedCols=["TECHNOLOGIES","energyCost","capacity","EnergyNbhourCap","_RampConstraintPlus","_RampConstraintMoins"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Gestion-Simple_TECHNOLOGIES.csv',sep=';',decimal=',', index=False)

SelectedCols=["TECHNOLOGIES","energyCost","capacity","EnergyNbhourCap","_RampConstraintPlus","_RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Gestion-RAMP1_TECHNOLOGIES.csv',sep=';',decimal=',', index=False)


### 3.3 Generating file for Planing <a class="anchor" id="3.3.Planing"></a>

In [224]:
ProductionTechPlus=ProductionTechPlus.assign(
    minCapacity = ProductionTechPlus["capacity"]*0.9,
    maxCapacity = ProductionTechPlus["capacity"]*100
)

SelectedCols=["TECHNOLOGIES","energyCost","capacityCost","EnergyNbhourCap",	"minCapacity","maxCapacity"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Planing-Simple_TECHNOLOGIES.csv',sep=';',decimal=',', index=False)

SelectedCols=["TECHNOLOGIES","energyCost","capacityCost","EnergyNbhourCap","minCapacity","maxCapacity","_RampConstraintPlus","_RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Planing-RAMP1_TECHNOLOGIES.csv',sep=';',decimal=',', index=False)



## 4. A first evaluation of System Cost <a class="anchor" id="3.SystemCost"></a>
Here we propose a first evaluation of system cost, it is similar to
 -  that proposed [in this post on the role of power and enery in system cost](https://www.energy-alternatives.eu/2020/05/07/mix-de-production-delectricite-energie-et-puissance.html).
 - and the one in an old excel file [here](https://docs.google.com/spreadsheets/d/1wWWu0YwsvkzmEG8XDRnsIoEegmALqZL-9XPxWx3z2Qo/edit?usp=sharing) contains similar stuff.

For that purpose, we only need
 - to use results from the preceding section (alpha and beta)
 - make assumptions of the installed power and yearly produced energy for each production mean.

For now we are not looking at how the equilibrium is done between production and consumption.

### 4.1 System cost computation <a class="anchor" id="3.2SystemCost"></a>

## 5. Multi-zone case <a class="anchor" id="5.multizone"></a>

### 5.1 Production data <a class="anchor" id="5.1.multizone"></a>

In [225]:
#region - general economic assumption
Production = pd.read_csv(InputFolder+'ProductionEconomicAssumptions.csv',sep=';',decimal=',',skiprows=0,comment="#")
qgrid_widget = qgrid.show_grid(Production, show_toolbar=True, column_definitions=col_def)
qgrid_widget

#endregion

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [226]:
Production=qgrid_widget.get_changed_df()
Production

,YEAR,AREAS,TECHNOLOGIES,CAPEX,dismantling,Variable,FOM,LL,CL,CO2Emission
0,2019,FR,TAC,600,100,60,36,30,2,600
1,2019,FR,Coal,1100,100,40,36,30,2,1000
2,2019,FR,CCG,900,100,50,36,30,2,470
3,2019,FR,OldNuke,0,1000,7,150,60,0,0
4,2019,FR,NukeCarrene,1587,1000,7,200,60,0,0
5,2019,FR,NewNuke,4000,1000,7,200,60,10,0
6,2019,FR,WindOnShore,1300,50,0,45,30,2,0
7,2019,FR,WindOffShore,2500,100,0,114,30,3,0
8,2019,FR,HydroLake,1500,10,0,20,80,10,0
9,2019,FR,HydroRiver,1500,10,0,20,80,10,0


In [227]:
r=(General.discountPercent[0]/100)
Production=Production.assign(
    LLr = round((1+r)/r*(1-(1+r)**(-Production['LL'])),2),
    capacityCost = round(Production.CAPEX/Production.LL+ Production.dismantling/(1+r)**(Production.LL) +Production.FOM,2),
    energyCost = round(Production.Variable + Production.CO2Emission*General.carbonTaxEurosPerT[0]/10**6,2)
)

Production
## Storage economic assumptions

,YEAR,AREAS,TECHNOLOGIES,CAPEX,dismantling,Variable,FOM,LL,CL,CO2Emission,LLr,capacityCost,energyCost
0,2019,FR,TAC,600,100,60,36,30,2,600,17.98,86.83,60.02
1,2019,FR,Coal,1100,100,40,36,30,2,1000,17.98,103.50,40.04
2,2019,FR,CCG,900,100,50,36,30,2,470,17.98,96.83,50.02
3,2019,FR,OldNuke,0,1000,7,150,60,0,0,23.53,245.06,7.00
4,2019,FR,NukeCarrene,1587,1000,7,200,60,0,0,23.53,321.51,7.00
5,2019,FR,NewNuke,4000,1000,7,200,60,10,0,23.53,361.73,7.00
6,2019,FR,WindOnShore,1300,50,0,45,30,2,0,17.98,103.75,0.00
7,2019,FR,WindOffShore,2500,100,0,114,30,3,0,17.98,228.17,0.00
8,2019,FR,HydroLake,1500,10,0,20,80,10,0,24.87,39.18,0.00
9,2019,FR,HydroRiver,1500,10,0,20,80,10,0,24.87,39.18,0.00


In [228]:
#region - general economic assumption
ProductionTech = pd.read_csv(InputFolder+'ProductionTechnicalAssumptions.csv',sep=';',decimal=',',skiprows=0,comment="#")
qgrid_widget = qgrid.show_grid(ProductionTech, show_toolbar=True, column_definitions=col_def)
qgrid_widget
ProductionTech=qgrid_widget.get_changed_df()
ProductionTechPlus=ProductionTech.merge(Production, how='inner', left_on=["AREAS","TECHNOLOGIES"], right_on=["AREAS","TECHNOLOGIES"])
ProductionTechPlus
#endregion


,YEAR_x,AREAS,TECHNOLOGIES,capacity,ProducedEnergy,EnergyNbhourCap,_RampConstraintPlus,_RampConstraintMoins,RampConstraintPlus2,RampConstraintMoins2,...,CAPEX,dismantling,Variable,FOM,LL,CL,CO2Emission,LLr,capacityCost,energyCost
0,2019,FR,TAC,4000.0,3.2,0.0,0.0,0.0,0,0,...,600,100,60,36,30,2,600,17.98,86.83,60.02
1,2019,FR,Coal,2000.0,1.0,0.0,0.1,0.1,0,0,...,1100,100,40,36,30,2,1000,17.98,103.50,40.04
2,2019,FR,CCG,12000.0,38.6,0.0,0.1,0.1,0,0,...,900,100,50,36,30,2,470,17.98,96.83,50.02
3,2019,FR,OldNuke,63000.0,413.0,0.0,0.1,0.1,0,0,...,0,1000,7,150,60,0,0,23.53,245.06,7.00
4,2019,FR,NukeCarrene,0.0,0.0,0.0,0.0,0.0,0,0,...,1587,1000,7,200,60,0,0,23.53,321.51,7.00
5,2019,FR,NewNuke,0.0,0.0,0.0,0.0,0.0,0,0,...,4000,1000,7,200,60,10,0,23.53,361.73,7.00
6,2019,FR,WindOnShore,14900.0,29.0,0.0,0.0,0.0,0,0,...,1300,50,0,45,30,2,0,17.98,103.75,0.00
7,2019,FR,WindOffShore,0.0,0.0,0.0,0.0,0.0,0,0,...,2500,100,0,114,30,3,0,17.98,228.17,0.00
8,2019,FR,HydroLake,12500.0,33.0,3638.1,0.0,0.0,0,0,...,1500,10,0,20,80,10,0,24.87,39.18,0.00
9,2019,FR,HydroRiver,7500.0,30.0,0.0,0.0,0.0,0,0,...,1500,10,0,20,80,10,0,24.87,39.18,0.00


### 5.2 Multizone Operation <a class="anchor" id="5.2.multizone"></a>

In [229]:
SelectedCols=["AREAS","TECHNOLOGIES","energyCost","capacity","EnergyNbhourCap","_RampConstraintPlus","_RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Gestion_MultiNode_DE-FR_AREAS_TECHNOLOGIES.csv',sep=';',decimal=',', index=False)

### 5.3 Multizone Planing <a class="anchor" id="5.3.multizone"></a>


In [230]:
ProductionTechPlus=ProductionTechPlus.assign(
    minCapacity = ProductionTechPlus["capacity"]*0.9,
    maxCapacity = ProductionTechPlus["capacity"]*100
)
SelectedCols=["AREAS","TECHNOLOGIES","energyCost","capacityCost","EnergyNbhourCap","minCapacity","maxCapacity","_RampConstraintPlus","_RampConstraintMoins","RampConstraintPlus2","RampConstraintMoins2"]
ProductionTechPlus[SelectedCols].to_csv(InputFolder+'Planing_MultiNode_DE-FR_TECHNOLOGIES_AREAS.csv',sep=';',decimal=',', index=False)